<a href="https://colab.research.google.com/github/pmopedro/computational-chess/blob/fabio/notebooks/chess_data_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importação de bibliotecas

In [1]:
!pip install python-chess
import chess.pgn
import io
import pandas as pd
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.9 MB/s eta 0:00:00


### Leitura dos dados de treinamento

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Leitura dos dados das partidas de xadrez
train_folder = '/content/gdrive/MyDrive/IA006_Projeto/train/'
datainfo = train_folder + 'chessData.csv'

df_train = pd.read_csv(datainfo)

In [4]:
df_train

,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-10
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,+56
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,-9
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,+52
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,-26
...,...,...
12958030,r1bqkb1r/pp3ppp/1nn1p3/3pP3/3P1P2/1B3N2/PP2Q1P...,+6
12958031,r2qkb1r/pp1b1ppp/1nn1p3/3pP3/3P1P2/1B3N2/PP2Q1...,+84
12958032,r2qkb1r/pp1b1ppp/1nn1p3/3pP3/3P1P2/1BN2N2/PP2Q...,0
12958033,r2qkb1r/pp1b1ppp/1n2p3/n2pP3/3P1P2/1BN2N2/PP2Q...,+115


### Leitura dos dados de teste

In [5]:
test_folder = '/content/gdrive/MyDrive/IA006_Projeto/test/'
test_info = test_folder + 'games.csv'

df_test = pd.read_csv(test_info)

In [6]:
df_test = df_test[['id', 'turns', 'winner', 'white_rating', 'black_rating', 'moves']]

In [7]:
# Dropa jogos com menos de 40 lances.
df_test = df_test[df_test.turns >= 40]

In [8]:
df_test

,id,turns,winner,white_rating,black_rating,moves
2,mIICvQHh,61,white,1496,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...
3,kWKvrqYL,61,white,1439,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...
4,9tXo1AUZ,95,white,1523,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...
8,dwF3DJHO,66,black,1439,1392,e4 e5 Bc4 Nc6 Nf3 Nd4 d3 Nxf3+ Qxf3 Nf6 h3 Bc5...
9,afoMwnLg,119,white,1381,1209,e4 d5 exd5 Qxd5 Nc3 Qe5+ Be2 Na6 d4 Qf5 Bxa6 b...
...,...,...,...,...,...,...
20049,7IENcPg3,43,white,1328,1252,e4 e6 Nf3 d5 Nc3 Bb4 exd5 exd5 d4 Bg4 a3 Bxc3+...
20051,uMzb0TPC,58,black,1237,1231,e4 e6 Nf3 d5 Bb5+ Bd7 c4 c6 Ba4 Qa5 b3 b5 cxb5...
20054,WSJDhbPl,82,black,1233,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...
20056,b0v4tRyF,109,white,1360,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...


#### Conversão de PGN para FEN após 40 lances (média geral de jogos de xadrez)

In [9]:
board_position = []
for game_moves in df_test['moves']:
  pgn = io.StringIO(game_moves)
  game = chess.pgn.read_game(pgn)

  board = game.board()
  move_cnt = 0
  for move in game.mainline_moves():
    board.push(move)
    move_cnt = move_cnt + 1
    if (move_cnt == 40): break

  board_position.append(board.fen())

In [10]:
# Adiciona lista de posições no dataframe.
se = pd.Series(board_position)
df_test['position'] = se.values

In [11]:
# Dropa a coluna dos movimentos, não é mais necessária.
df_test = df_test.drop('moves', axis=1)

In [12]:
# Exporta o dataframe para um novo arquivo csv no Drive, na pasta de teste.
test_output = test_folder + 'games_positions.csv'
df_test.to_csv(test_output)

In [13]:
df_test

,id,turns,winner,white_rating,black_rating,position
2,mIICvQHh,61,white,1496,1500,R5nr/5p1p/2P1P2k/4p3/8/2P5/1P2BPPP/3QK1NR w K ...
3,kWKvrqYL,61,white,1439,1454,1Q3b1r/4kppp/2p5/3pP3/4b1n1/4PN2/1P2BPPP/R4RK1...
4,9tXo1AUZ,95,white,1523,1469,2r3k1/pp1b1ppp/3p1q2/1P2p3/P3P3/3Q1N2/2P2PPP/R...
8,dwF3DJHO,66,black,1439,1392,2r1r1k1/p4ppp/1q1p1n2/2pP4/2P1p3/PB3Q1P/1P3PP1...
9,afoMwnLg,119,white,1381,1209,2r2b1r/p4k1p/N1p1pnp1/8/3P1Bb1/6R1/PPP2P1P/R3K...
...,...,...,...,...,...,...
20049,7IENcPg3,43,white,1328,1252,6k1/ppp2ppp/2n2r2/3p4/3P2P1/P4P1q/2PQ3P/5RK1 w...
20051,uMzb0TPC,58,black,1237,1231,r3k2r/p4ppp/2n5/qp2p3/6P1/2N1b3/PB4PP/n2QK2R w...
20054,WSJDhbPl,82,black,1233,1196,rn4k1/p1p2ppp/1p5r/3p2q1/3P4/2PQ4/PP3PN1/R5KR ...
20056,b0v4tRyF,109,white,1360,1227,2kr3r/pp4pp/2p2b2/2P5/1PNpP1P1/8/P6P/R2K3R w -...
